# Scraping sunset sunrise

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

### Open driver

In [ ]:
url = 'https://www.esrl.noaa.gov/gmd/grad/solcalc/sunrise.html'
lat = [40.53]
lon = [-20.3]
driver = webdriver.Chrome('C:/chromedriver.exe')
driver.get(url)
months = ['January','February','March','April','May','June','July','August','September','October','November','December']

### Conversion lat lon decimal to degree

In [3]:
def conv(lat):
    deg = int(lat)
    min_ = abs(int(int((lat-int(lat))*100)*0.6))
    sec = int(int((min_-int(min_))*100)*0.6)
    return deg, min_, sec
conv(-20.3)

(-20, 18, 0)

### Function taking the average hour

In [ ]:
def mean_hour(L):
    h = []
    for s in L:
        if s[2] == ':':
            h.append(int(s[0:2])+int(s[3:5])/60)
            pa = s[5:7]
        else:
            h.append(int(s[0])+int(s[2:4])/60)
            pa = s[4:6]
    m = sum(h)/len(h)
    mean = str(int(m))+':'+str(int(int((m-int(m))*100)*0.6))+pa
    return mean

def mean_hour_utc(L):
    h = []
    for s in L:
        if s[2] == ':':
            h.append(int(s[0:2])+int(s[3:5])/60)
        else:
            h.append(int(s[0])+int(s[2:4])/60)
    m = sum(h)/len(h)
    mean = str(int(m))+':'+str(int(int((m-int(m))*100)*0.6))
    return mean
min_hour_utc(sunrise[0][2])

### Scraping noaa

In [5]:
driver.find_element(By.XPATH,"//select[@name='cities']/option").click()
output = pd.DataFrame()
for i in range(len(lat)):
    lat_deg, lat_min, lat_sec = conv(lat[i])
    lon_deg, lon_min, lon_sec = conv(lon[i])
    latDeg = driver.find_element(By.XPATH,"//input[@name = 'latDeg']")
    latDeg.clear()
    latDeg.send_keys(lat_deg)
    latMin = driver.find_element(By.XPATH,"//input[@name = 'latMin']")
    latMin.clear()
    latMin.send_keys(lat_min)
    latSec = driver.find_element(By.XPATH,"//input[@name = 'latSec']")
    latSec.clear()
    latSec.send_keys(lat_sec)
    lonDeg = driver.find_element(By.XPATH,"//input[@name = 'lonDeg']")
    lonDeg.clear()
    lonDeg.send_keys(lon_deg)
    lonMin = driver.find_element(By.XPATH,"//input[@name = 'lonMin']")
    lonMin.clear()
    lonMin.send_keys(lon_min)
    lonSec = driver.find_element(By.XPATH,"//input[@name = 'lonSec']")
    lonSec.clear()
    lonSec.send_keys(lon_sec)
    mean_sunrise = []
    mean_sunset = []
    mean_sunrise_utc = []
    mean_sunset_utc =[]
    lat = [lat[i] for k in range(12)]
    lon = [lon[i] for k in range(12)]
    new_df = pd.DataFrame()
    for month in months:        
        if month == 'February':
            m = 28
        else:
            m = 30
        up_local = []
        up_utc = []
        down_local = []
        down_utc = []
        for i in range(m):
            driver.find_element(By.XPATH,"//input[@name='day']").clear()
            driver.find_element(By.XPATH,"//input[@name='day']").send_keys(i+1)
            driver.find_element_by_xpath("//select[@name='mos']/option[contains(text(), '" + month + "')]").click()
            driver.find_element(By.XPATH,"//input[@name='RiseSet']").click()
            up_local.append(driver.find_element(By.XPATH,"//input[@name='sunrise']").get_attribute('value'))
            up_utc.append(driver.find_element(By.XPATH,"//input[@name='utcsunrise']").get_attribute('value'))
            down_local.append(driver.find_element(By.XPATH,"//input[@name='sunset']").get_attribute('value'))
            down_utc.append(driver.find_element(By.XPATH,"//input[@name='utcsunset']").get_attribute('value'))
        mean_sunrise.append(mean_hour(up_local))
        mean_sunset.append(mean_hour(down_local))
        mean_sunrise_utc.append(mean_hour_utc(up_utc))
        mean_sunset_utc.append(mean_hour_utc(down_utc))
    new_df['LATITUDE'] = lat
    new_df['LONGITUDE'] = lon
    new_df['MONTH'] = months
    new_df['MEAN SUNRISE'] = mean_sunrise
    new_df['MEAN SUNRISE UTC'] = mean_sunrise_utc
    new_df['MEAN SUNSET'] = mean_sunset
    new_df['MEAN SUNSET UTC'] = mean_sunset_utc
    output = pd.concat([output,new_df])
driver.close()   

### Writing to csv

In [93]:
output.to_csv('C:/users/antoi/desktop/Gates_data/sunset_sunrise/sample.csv')

In [94]:
output

,LATITUDE,LONGITUDE,MONTH,MEAN SUNRISE,MEAN SUNRISE UTC,MEAN SUNSET,MEAN SUNSET UTC
0,40.53,-20.3,January,10:58PM,5:58,8:37AM,15:37
1,40.53,-20.3,February,10:32PM,5:32,9:12AM,16:12
2,40.53,-20.3,March,9:49PM,4:49,9:45AM,16:45
3,40.53,-20.3,April,9:0PM,4:0,10:18AM,17:18
4,40.53,-20.3,May,8:22PM,3:22,10:48AM,17:48
5,40.53,-20.3,June,8:8PM,3:8,11:9AM,18:9
6,40.53,-20.3,July,8:21PM,3:21,11:6AM,18:6
7,40.53,-20.3,August,8:49PM,3:49,10:35AM,17:34
8,40.53,-20.3,September,9:19PM,4:19,9:46AM,16:46
9,40.53,-20.3,October,9:49PM,4:49,8:58AM,15:58
